In [1]:
import spacy
import json

In [2]:
# load the pipeline for english
nlp = spacy.load('en')

#### load all the corpus we have so that we can find what we want

In [3]:
with open('js_corpus.json') as js_corpus_json:
    js_corpus = json.load(js_corpus_json)
with open('compare_corpus.json') as compare_corpus_json:
    c_corpus = json.load(compare_corpus_json)

In [4]:
js_corpus_without_noun = []
c_corpus_without_noun = []

#### At this point we have following variables
* js_corpus / c_corpus : loaded json corpus
* js_corpus_without_noun / c_corpus_without_noun : corpus without noun tokens
* js_corpus_tokenized / c_corpus_tokenized : tokenized form of all the text in question

## Data Cleaning

In [5]:
# remove all the blank spaces and new lines,
# in front and in back of the qustion text
js_corpus = [question['sQuestion'].strip('\n ') 
             for question in js_corpus]
c_corpus = [question['sQuestion'].strip('\n ')
           for question in c_corpus]

In [6]:
js_corpus_tokenized = [list(nlp(question))
                       for question in js_corpus]
c_corpus_tokenized = [list(nlp(question)) 
                      for question in c_corpus]

In [7]:
# list of all the unnecessesary tokens with following pos tags
pos_nouns_list = ["NOUN", "PROPN", "DET", "PUNCT", "PRON"]
tag_adjs_list = ["PRP$", "WP$", "SPACE"]

In [8]:
for i in range(len(js_corpus_tokenized)):
    js_corpus_without_noun.append([token for token in js_corpus_tokenized[i] 
                                   if (token.pos_ not in pos_nouns_list) and (token.tag_ not in tag_adjs_list)])

for i in range(len(c_corpus_tokenized)):
    c_corpus_without_noun.append([token for token in c_corpus_tokenized[i]
                                  if (token.pos_ not in pos_nouns_list) and (token.tag_ not in tag_adjs_list)])

In [9]:
# random check if we miss some tokens
q0_js_doc = [token.orth_ for token in js_corpus_without_noun[2]]
print(q0_js_doc)

q0_c_doc = [token.orth_ for token in c_corpus_without_noun[4]]
print(q0_c_doc)

['had', 'gave', '4', 'more', 'Then', 'had', '6', 'How', 'many', 'did', 'start', 'with']
['has', '13', 'has', '9', 'How', 'many', 'more', 'does', 'have', 'than']


### Above output looks good as there is no unwanted data string is present, but need to check if removing HOW works or not 

# Pickling everything 

In [48]:
js_vectors = [np.array([np.array(token.vector, dtype=np.float64) for token in js_corpus_without_noun[i]], dtype=np.float64) 
               for i in range(len(js_corpus_without_noun))]

In [49]:
c_vectors = [np.array([np.array(token.vector, dtype=np.float64) for token in c_corpus_without_noun[i]], dtype=np.float64) 
             for i in range(len(c_corpus_without_noun))]

In [13]:
import pickle as pkl

In [15]:
# Dumping all the vectors data to file
with open('js_vectors.pickle','wb') as f: 
    pkl.dump(js_vectors, f)

In [16]:
with open('c_vectors.pickle','wb') as f: 
    pkl.dump(c_vectors, f)

##### Loading the data
`pickle_in = open('linearregression.pickle','rb') clf = pickle.load(pickle_in)`

# Cross Product with numpy and then pickling all the results

In [19]:
import numpy as np
import functools as fts

### Define new var :
`js_with_single_vector_question_representation`
`c_with_single_vector_question_representation`

In [20]:
js_with_1_vec_que_rep = []
c_with_1_vec_que_rep = []

# numpy NOT WORKING !!! 

In [25]:
import tensorflow as tf

In [27]:
init_arr = np.array([i for i in range(300)], dtype=np.float32)
# two variables 
x1 = tf.Variable(init_arr, name="x1")
x2 = tf.Variable(init_arr, name="x2")

In [28]:
model = tf.global_variables_initializer()
session = tf.InteractiveSession()

In [29]:
session.run(model)

In [33]:
js_with_1_vec_que_rep = [fts.reduce(lambda x, y: session.run(tf.cross(x1.assign(x), x2.assign(y))), js_vectors[i]) for i in range(len(js_vectors))]

FailedPreconditionError: Cross-products are only defined for 3-element vectors.
	 [[Node: Cross = Cross[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](Assign, Assign_1)]]

Caused by op 'Cross', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/lib/python3.6/site-packages/tornado/ioloop.py", line 887, in start
    handler_func(fd_obj, events)
  Fi

# Trying to put TSNE to use and reducing the dimentionality of the each word which then can be cross produced to find the vector for the document

In [34]:
from sklearn.manifold import TSNE
model = TSNE(n_components=3, random_state=0)

In [50]:
model.fit(np.array(js_vectors, dtype=np.float64), y=None)

ValueError: setting an array element with a sequence.

In [55]:
X = np.array([tok.vector for tok in js_corpus_without_noun[i]])

In [56]:
model.fit_transform(X)

array([[  1.66441886e-04,   3.98365451e-05,   8.53404605e-05],
       [  2.07754975e-04,   1.69320718e-04,  -8.73236896e-05],
       [  9.42024317e-05,  -9.29142895e-06,  -1.03560104e-05],
       [  4.64018224e-05,   1.72688736e-05,   1.26986044e-04],
       [  7.75159938e-05,   1.51233991e-05,   3.79123471e-05],
       [  3.97611370e-05,   1.36843390e-04,  -1.95387804e-05],
       [  3.81232305e-05,  -6.99385521e-05,  -2.24646041e-04],
       [  6.78643380e-05,   8.10484252e-05,  -6.70807102e-05],
       [  2.10309557e-04,  -1.22973783e-04,   2.83611269e-06],
       [ -5.87903268e-06,   1.39363733e-04,   1.27665502e-04],
       [  2.33770926e-05,   3.80972692e-05,  -7.97475887e-05]])

In [57]:
js_vectors = [model.fit_transform(que) for que in js_vectors]

In [58]:
c_vectors = [model.fit_transform(que) for que in c_vectors]

In [59]:
with open('js_vectors_tnse.pickle','wb') as f: 
    pkl.dump(js_vectors, f)
    
with open('c_vectors_tnse.pickle', 'wb') as f:
    pkl.dump(c_vectors, f)

In [60]:
js_ques_vectors = [fts.reduce(lambda x, y: np.cross(x, y), js_vectors[i]) for i in range(len(js_vectors))]

In [61]:
c_ques_vecrors = [fts.reduce(lambda x, y: np.cross(x, y), c_vectors[i]) for i in range(len(c_vectors))]

In [71]:
print(js_ques_vectors[0])

[  6.41858991e-40  -2.29518588e-40   2.68870594e-40]


In [73]:
with open('js_crossed_que_vecs.pickle', 'wb') as f:
    pkl.dump(js_ques_vectors, f)
    
with open('c_crossed_que_vecs.pickle', 'wb') as f:
    pkl.dump(c_ques_vecrors, f)